## Separar datasets

In [1]:
import pandas as pd
import numpy as np
fn = './datasets/Altura_y_Caudal_Instantaneo_Diario_19_05_2022_13_43.xls'
fn2 = './datasets/Altura_y_Caudal Instantáneo_(Diario)2015-2017.xls'
fn3= './datasets/Altura_y_Caudal_Instantáneo_(Diario)2019-2021.xls'
fn4 = './datasets/Altura_y_Caudal_Instantáneo(Diario)2017-2019.xls'

In [2]:
COLUMNAS = ['DIA', 'HORA', 'ALTURA (m)', 'CAUDAL (m3/seg)', 'I', 'O']
MESES = [
    "ENERO",
    "FEBRERO",
    "MARZO",
    "ABRIL",
    "MAYO",
    "JUNIO",
    "JULIO",
    "AGOSTO",
    "SEPTIEMBRE",
    "OCTUBRE",
    "NOVIEMBRE",
    "DICIEMBRE"
]

files = [fn,fn2,fn3,fn4]

In [3]:
def unir_columnas_repetidas(df,index):
    df_1 = df.iloc[:,:6]
    df_2 = df.iloc[:,6:12]
    df_3 = df.iloc[:,12:18]
    fecha = df.iloc[:,18:]
    df_1 = pd.concat([df_1,fecha],axis=1)
    df_2.columns = COLUMNAS
    df_2 = pd.concat([df_2,fecha],axis=1)
    df_3.columns = COLUMNAS
    df_3 = pd.concat([df_3,fecha],axis=1)
    return df_1,df_2,df_3

In [4]:
for excel in files:
    xl = pd.ExcelFile(excel)
    for index,sheet in enumerate(range(len(xl.sheet_names))):
        print(index)

        df11 = xl.parse(0, skiprows=10,).dropna(axis=1, how='all') # Posee primer año y mes
        df = xl.parse(index, skiprows=11,).dropna(axis=1, how='all') # Tablas
        MES = int(df11.columns[2].split("/")[0])
        AÑO = int(df11.columns[2].split("/")[1])

        del df11

        sep = df[df.isna().any(axis=1)]
        sep = sep[(sep['DIA']) == 'MES:']
        datasets = [[] for _ in range(len(sep)+1)]
        j=0

        for i,row in df.iterrows():
            if (row['DIA'] == 'MES:'):
                if(j==len(sep)+1):
                    break
                j+=1
                mes = int(row['ALTURA (m)'].split("/")[0])
                ano = int(row['ALTURA (m)'].split("/")[1])
                continue
            else:
                if((row['DIA'] == 'DIA') or row['DIA']== 'Indicadores'):
                    continue
                else:
                    try:
                        row['MES'] = mes
                        row['AÑO'] = ano
                        datasets[j].append(row)
                    except NameError:
                        row['MES'] = MES
                        row['AÑO'] = AÑO
                        datasets[j].append(row)

        for index2,dataset in enumerate(datasets):
            df = pd.DataFrame(dataset)
            df_1,df_2,df_3 = unir_columnas_repetidas(df,index)
            if index2==0:
                dfx = pd.concat([df_1,df_2,df_3],axis=0)
            else:
                dfx = pd.concat([dfx,df_1,df_2,df_3],axis=0)

        #Ordenamos por Hora,mes año, dia
        dfx = dfx.sort_values(['AÑO','MES','DIA','HORA'],ascending=[True,True,True,True])

#         #Asignamos MES como palabra
        for position,row in dfx.iterrows():
            dfx.at[position,'MES'] = MESES[row['MES']-1]


        #Guardado
        dfx.to_excel(f"./processed_datasets/{xl.sheet_names[index]}_{AÑO}-{ano}.xlsx",index = False)
        print("creado",xl.sheet_names[index])
        del mes
        del ano
        del MES
        del AÑO
print("fin")

0
creado RIO ACONCAGUA EN CHACABUQUITO
1
creado RIO ACONCAGUA EN SAN FELIPE
0
creado RIO ACONCAGUA EN CHACABUQUITO
1
creado RIO ACONCAGUA EN SAN FELIPE
0
creado RIO ACONCAGUA EN CHACABUQUITO
1
creado RIO ACONCAGUA EN SAN FELIPE
0
creado RIO ACONCAGUA EN CHACABUQUITO
1
creado RIO ACONCAGUA EN SAN FELIPE
